In [1]:
import pandas as pd
from mwclient import Site
import json 
import requests
site = Site('lol.fandom.com', path="/")

tournament = "LCK/2024 Season/Spring Season"
team = 'Gen.G'

### Get split matches from a given team and split. Can return Game end and Game timeline

In [2]:
fields = [
    "MSG.RiotPlatformGameId",
    "MS.ShownName",
    "MS.DateTime_UTC",
    "MSG.Blue",
    "MSG.Red",
    "MSG.N_GameInMatch",
    "MS.BestOf",
    "MSG.Winner",
    "PAB.Team1PicksByRoleOrder",
    "PAB.Team2PicksByRoleOrder",
   
]

cargo_query = site.api(
    "cargoquery",
    limit = "max",
    tables = "MatchSchedule = MS,MatchScheduleGame = MSG, PicksAndBansS7 = PAB",
    fields = ",".join(fields),
    where = 'MS.OverviewPage="{}"'.format(tournament) + 'OR MSG.Blue ="{}"'.format(team) +'OR MSG.Red ="{}"'.format(team),
    order_by = "MS.DateTime_UTC DESC",
    join_on = "MS.MatchId = MSG.MatchId, MSG.GameId = PAB.GameId"
)


In [3]:
df = pd.DataFrame([d['title'] for d in cargo_query['cargoquery']])
df_spring24 = df[df['ShownName'].isin(['LCK 2024 Spring','LCK 2024 Spring Playoffs'])]
df_spring24 = df_spring24[(df_spring24['Blue'] == 'Gen.G') | (df_spring24['Red'] == 'Gen.G')]
print(len(df_spring24))
df_spring24.head()


53


,RiotPlatformGameId,ShownName,DateTime UTC,Blue,Red,N GameInMatch,BestOf,Winner,Team1PicksByRoleOrder,Team2PicksByRoleOrder,DateTime UTC__precision
0,LOLTMNT02_76144,LCK 2024 Spring Playoffs,2024-04-14 06:00:00,Gen.G,T1,1,5,1,"Rumble,Viego,Corki,Kalista,Nautilus","Aatrox,Maokai,Azir,Draven,Poppy",0
1,LOLTMNT02_76145,LCK 2024 Spring Playoffs,2024-04-14 06:00:00,T1,Gen.G,2,5,1,"K'Sante,Xin Zhao,Azir,Kalista,Neeko","Rek'Sai,Rell,Aurelion Sol,Varus,Annie",0
2,LOLTMNT02_76146,LCK 2024 Spring Playoffs,2024-04-14 06:00:00,Gen.G,T1,3,5,2,"Rek'Sai,Sejuani,Azir,Aphelios,Lulu","Zac,Xin Zhao,Taliyah,Lucian,Nami",0
3,LOLTMNT02_76147,LCK 2024 Spring Playoffs,2024-04-14 06:00:00,Gen.G,T1,4,5,1,"K'Sante,Kha'Zix,Aurelion Sol,Lucian,Nami","Gwen,Xin Zhao,Azir,Aphelios,Lulu",0
4,LOLTMNT02_76148,LCK 2024 Spring Playoffs,2024-04-14 06:00:00,T1,Gen.G,5,5,2,"Zac,Xin Zhao,Orianna,Lucian,Nami","K'Sante,Poppy,Corki,Zeri,Annie",0


In [4]:
# get the game data from leaguepedia (game_end, timeline) using the RiotPlatformGameId as input
def game_format(id_game):
    gameDatas = json.loads(requests.get(f"https://lol.fandom.com/api.php?action=query&format=json&prop=revisions&titles=V5%20data%3A{id_game}&rvprop=content&rvslots=main").content)
    gameDatas = json.dumps(gameDatas)
    gameDatas = gameDatas.replace(' ', '')
    gameDatas = gameDatas.replace('\\n', '')
    gameDatas = json.loads(gameDatas)
    for y in gameDatas['query']['pages']:
        gameDatas = gameDatas['query']['pages'][y]['revisions'][0]['slots']['main']['*']
    gameDatas = json.loads(gameDatas)

    gameTimeline = json.loads(requests.get(f"https://lol.fandom.com/api.php?action=query&format=json&prop=revisions&titles=V5%20data%3A{id_game}%2FTimeline&rvprop=content&rvslots=main").content)
    gameTimeline = json.dumps(gameTimeline)
    gameTimeline = gameTimeline.replace(' ', '')
    gameTimeline = gameTimeline.replace('\\n', '')
    gameTimeline = json.loads(gameTimeline)
    for y in gameTimeline['query']['pages']:
        gameTimeline = gameTimeline['query']['pages'][y]['revisions'][0]['slots']['main']['*']
    gameTimeline = json.loads(gameTimeline)

    return gameDatas,gameTimeline

In [5]:
list_game_end = []
list_game_timeline = []
for game_id in df_spring24['RiotPlatformGameId'].unique().tolist():
    game_end,game_timeline = game_format(game_id)
    list_game_end.append(game_end)
    list_game_timeline.append(game_timeline)
